# Scenario
**Chicago Airbnb**

You and a group of friends are
considering purchasing a property in
Chicago that you can use as an
investment. You have heard from other
people that they have made a lot of
money by renting out either a room or
an entire unit (apartment or house). Your
friends ask you to analyze data so that
they can understand how much you
would charge per night based on the
type of dwelling you were to purchase.

**Dataset:**
https://www.kaggle.com/datasets/jinbonnie/chicago-airbnb-open-data

Now that the data has been prepared, I will make the necessary transformations to the data so that it can be used in a machine learning model. Then I will train a model to predict the 'availability_365'.

# Imports

In [5]:
import numpy as np
import pandas as pd
import joblib
import numpy as np
import pandas as pd
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import cross_val_score, cross_val_predict, GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.tree import DecisionTreeRegressor
from sqlalchemy import create_engine
from tqdm import tqdm
tqdm.pandas()

import credentials

# Loading cleaned data

In [6]:
db_con = create_engine(credentials.DB_URL)

train_df = pd.read_sql('SELECT * FROM cleaned.train_listings', db_con)
val_df = pd.read_sql('SELECT * FROM cleaned.val_listings', db_con)

train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5012 entries, 0 to 5011
Data columns (total 16 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   neighbourhood                 5012 non-null   object 
 1   latitude                      5012 non-null   float64
 2   longitude                     5012 non-null   float64
 3   room_type                     5012 non-null   object 
 4   price                         5012 non-null   int64  
 5   log_days_since_last_review    5012 non-null   float64
 6   log_reviews_per_month         5012 non-null   float64
 7   log_number_of_reviews         5012 non-null   float64
 8   log_price                     5012 non-null   float64
 9   log_minimum_nights            5012 non-null   float64
 10  log_host_listings_count       5012 non-null   float64
 11  log_nights_booked             5012 non-null   float64
 12  host_listings_minimum_nights  5012 non-null   float64
 13  pri

# Preprocessing pipeline

Here's the deal: the goal is to optimize the income from listings based on changing the price input into the model. Scaling the features adds a whole other layer of complexity to that optimization so to avoid that, I will only use models that are scale-invariant and not use scaling in the preprocessing pipeline. 

What I will do though is try OneHotEncoding vs. OrdinalEncoding for the categorical features and see which one performs better. I know OrdinalEncoding injects some bias, but I want to try, because otherwise features like neighbourhood will be made quite sparse.

In [7]:
# Split data into X and y
X_train, y_train = train_df.drop(columns='log_nights_booked'), train_df['log_nights_booked']
X_val, y_val = val_df.drop(columns='log_nights_booked'), val_df['log_nights_booked']

# Set objects to category
X_train['neighbourhood'] = X_train['neighbourhood'].astype('category')
X_val['neighbourhood'] = X_val['neighbourhood'].astype('category')
X_train['room_type'] = X_train['room_type'].astype('category')
X_val['room_type'] = X_val['room_type'].astype('category')

In [8]:
ohe_processor = make_column_transformer(
    (OneHotEncoder(handle_unknown='infrequent_if_exist'), make_column_selector(dtype_include='category')),
    remainder='passthrough'
)
ord_processor = make_column_transformer(
    (OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), make_column_selector(dtype_include='category')),
    remainder='passthrough'
)
pd.DataFrame(ord_processor.fit_transform(X_train), columns=ord_processor.get_feature_names_out())

,ordinalencoder__neighbourhood,ordinalencoder__room_type,remainder__latitude,remainder__longitude,remainder__price,remainder__log_days_since_last_review,remainder__log_reviews_per_month,remainder__log_number_of_reviews,remainder__log_price,remainder__log_minimum_nights,remainder__log_host_listings_count,remainder__host_listings_minimum_nights,remainder__price^2,remainder__price^3,remainder__price^4
0,29.0,2.0,41.64736,-87.61679,69.0,-1.000000,-1.000000,-1.000000,1.838849,0.000000,0.00000,0.000000,4761.0,328509.0,2.266712e+07
1,29.0,2.0,41.65208,-87.61030,22.0,-1.000000,-1.000000,-1.000000,1.342423,1.477121,0.00000,0.000000,484.0,10648.0,2.342560e+05
2,29.0,2.0,41.65388,-87.61021,22.0,-1.000000,-1.000000,-1.000000,1.342423,1.477121,0.00000,0.000000,484.0,10648.0,2.342560e+05
3,29.0,0.0,41.65578,-87.53790,200.0,2.272074,-0.494850,0.612784,2.301030,1.778151,0.60206,1.070554,40000.0,8000000.0,1.600000e+09
4,29.0,0.0,41.65977,-87.65236,90.0,-1.000000,-1.000000,-1.000000,1.954243,0.000000,0.60206,0.000000,8100.0,729000.0,6.561000e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5007,23.0,2.0,42.02087,-87.67286,68.0,2.441066,-0.568636,0.041393,1.832509,0.698970,0.30103,0.210411,4624.0,314432.0,2.138138e+07
5008,23.0,2.0,42.02105,-87.67300,45.0,2.114277,0.250420,1.903633,1.653213,0.477121,0.30103,0.143628,2025.0,91125.0,4.100625e+06
5009,23.0,0.0,42.02119,-87.66411,140.0,2.025715,0.367356,1.149219,2.146128,0.477121,0.00000,0.000000,19600.0,2744000.0,3.841600e+08
5010,23.0,2.0,42.02139,-87.67363,60.0,2.541704,-0.040959,1.725095,1.778151,0.477121,0.30103,0.143628,3600.0,216000.0,1.296000e+07


In [9]:
pd.DataFrame(ohe_processor.fit_transform(X_train), columns=ohe_processor.get_feature_names_out())

,onehotencoder__neighbourhood_Albany Park,onehotencoder__neighbourhood_Armour Square,onehotencoder__neighbourhood_Austin,onehotencoder__neighbourhood_Avondale,onehotencoder__neighbourhood_Bridgeport,onehotencoder__neighbourhood_Douglas,onehotencoder__neighbourhood_East Garfield Park,onehotencoder__neighbourhood_Edgewater,onehotencoder__neighbourhood_Grand Boulevard,onehotencoder__neighbourhood_Humboldt Park,onehotencoder__neighbourhood_Hyde Park,onehotencoder__neighbourhood_Irving Park,onehotencoder__neighbourhood_Lake View,onehotencoder__neighbourhood_Lincoln Park,onehotencoder__neighbourhood_Lincoln Square,onehotencoder__neighbourhood_Logan Square,onehotencoder__neighbourhood_Loop,onehotencoder__neighbourhood_Lower West Side,onehotencoder__neighbourhood_Near North Side,onehotencoder__neighbourhood_Near South Side,onehotencoder__neighbourhood_Near West Side,onehotencoder__neighbourhood_North Center,onehotencoder__neighbourhood_Portage Park,onehotencoder__neighbourhood_Rogers Park,onehotencoder__neighbourhood_South Shore,onehotencoder__neighbourhood_Uptown,onehotencoder__neighbourhood_West Ridge,onehotencoder__neighbourhood_West Town,onehotencoder__neighbourhood_Woodlawn,onehotencoder__neighbourhood_infrequent,onehotencoder__room_type_Entire home/apt,onehotencoder__room_type_Hotel room,onehotencoder__room_type_Private room,onehotencoder__room_type_Shared room,remainder__latitude,remainder__longitude,remainder__price,remainder__log_days_since_last_review,remainder__log_reviews_per_month,remainder__log_number_of_reviews,remainder__log_price,remainder__log_minimum_nights,remainder__log_host_listings_count,remainder__host_listings_minimum_nights,remainder__price^2,remainder__price^3,remainder__price^4
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,41.64736,-87.61679,69.0,-1.000000,-1.000000,-1.000000,1.838849,0.000000,0.00000,0.000000,4761.0,328509.0,2.266712e+07
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,41.65208,-87.61030,22.0,-1.000000,-1.000000,-1.000000,1.342423,1.477121,0.00000,0.000000,484.0,10648.0,2.342560e+05
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,41.65388,-87.61021,22.0,-1.000000,-1.000000,-1.000000,1.342423,1.477121,0.00000,0.000000,484.0,10648.0,2.342560e+05
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,41.65578,-87.53790,200.0,2.272074,-0.494850,0.612784,2.301030,1.778151,0.60206,1.070554,40000.0,8000000.0,1.600000e+09
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,41.65977,-87.65236,90.0,-1.000000,-1.000000,-1.000000,1.954243,0.000000,0.60206,0.000000,8100.0,729000.0,6.561000e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5007,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,42.02087,-87.67286,68.0,2.441066,-0.568636,0.041393,1.832509,0.698970,0.30103,0.210411,4624.0,314432.0,2.138138e+07
5008,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,42.02105,-87.67300,45.0,2.114277,0.250420,1.903633,1.653213,0.477121,0.30103,0.143628,2025.0,91125.0,4.100625e+06
5009,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,42.02119,-87.66411,140.0,2.025715,0.367356,1.149219,2.146128,0.477121,0.00000,0.000000,19600.0,2744000.0,3.841600e+08
5010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

Looks like it works.

# Ordinal vs OneHot

I like Random Forests, so that's what I'll use for the test of Ordinal vs OneHot. Most of the models I will try are tree based and Random Forests are a good baseline for that.

In [10]:
rfr = RandomForestRegressor(random_state=42)
ohe_pipe = make_pipeline(ohe_processor, rfr)
ord_pipe = make_pipeline(ord_processor, rfr)
ohe_scores = cross_val_score(ohe_pipe, X_train, y_train, cv=3, scoring='r2', n_jobs=-1)
ord_scores = cross_val_score(ord_pipe, X_train, y_train, cv=3, scoring='r2', n_jobs=-1)
print(f"OHE: {ohe_scores.mean()}")
print(f"ORD: {ord_scores.mean()}")

OHE: 0.025578241378462024
ORD: -0.09388595400477136


Well, it was worth a try. OneHotEncoding is what I will use for the categorical features. Those R2 scores are not boding well.

# Model selection

Again, only scale-invariant models. Mainly tree-based models and linear regression with no regularization.

In [ ]:
models = [
    # Linear Models
    ('Linear Regression', LinearRegression()),
    
    # Tree Models
    ('Decision Tree', DecisionTreeRegressor(random_state=42)),
    ('AdaBoost', AdaBoostRegressor(random_state=42)),
    ('Random Forest', RandomForestRegressor(random_state=42)),
    ('Gradient Boosting', GradientBoostingRegressor(random_state=42)),
]

model_names = []
MSEs = []
MAEs = []
r2_scores = []

for name, model in models:
    print(f'Running {name}')
    pipe = make_pipeline(ohe_processor, model)
    preds = cross_val_predict(pipe, X_train, y_train, cv=3, n_jobs=-1)
    
    # Append results to lists
    model_names.append(name)
    MSEs.append(mean_squared_error(y_train, preds))
    MAEs.append(mean_absolute_error(y_train, preds))
    r2_scores.append(r2_score(y_train, preds))

# Create DataFrame
results_df = pd.DataFrame({
    'Model': model_names,
    'MSE': MSEs,
    'MAE': MAEs,
    'R2': r2_scores
})
results_df.sort_values('R2', ascending=False)

Running Linear Regression
Running Decision Tree
Running AdaBoost
Running Random Forest
Running Gradient Boosting


,Model,MSE,MAE,R2
3,Random Forest,0.838758,0.716860,0.069879
4,Gradient Boosting,0.859349,0.682696,0.047045
2,AdaBoost,0.877313,0.784835,0.027124
0,Linear Regression,0.901441,0.704864,0.000368
1,Decision Tree,1.606044,0.836553,-0.780985


Random Forest Regressor is the best model for this dataset. Had to do some research as to why the Random Forest R2 score doesn't match above. The cross_val_score is getting the mean R2 from all the folds, while the one here is the R2 across the whole dataset from the cross_val_predict.

# Random Forest hyperparameter tuning

Random forests are scale invariant so I wont include the StandardScaler in the pipeline and I'll unscale the target.

In [15]:
pipe = make_pipeline(ohe_processor, rfr)
param_grid = {
    'randomforestregressor__max_depth': np.arange(3, 16, 3).tolist() + [None],
    'randomforestregressor__max_features': ['sqrt', None, 0.5],
}

rfr_search = GridSearchCV(pipe, param_grid, cv=3, scoring='r2', n_jobs=-1, verbose=1)
rfr_search.fit(X_train, y_train)

Fitting 3 folds for each of 18 candidates, totalling 54 fits


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('onehotencoder',
                                                                         OneHotEncoder(handle_unknown='infrequent_if_exist'),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x7b10fd75d2d0>)])),
                                       ('randomforestregressor',
                                        RandomForestRegressor(random_state=42))]),
             n_jobs=-1,
             param_grid={'randomforestregressor__max_depth': [3, 6, 9, 12, 15,
                                                              None],
                         'randomforestregressor__max_features': ['sqrt', None,
                                                                 0.5]},
             scoring='r2', verbose=1)

In [16]:
rfr_search.best_params_, rfr_search.best_score_

({'randomforestregressor__max_depth': 12,
  'randomforestregressor__max_features': 'sqrt'},
 np.float64(0.16176040003163636))

The score is better than the base hyperparameters, but still not great.

# Testing on the validation set

In [17]:
r2_score(y_val, rfr_search.best_estimator_.predict(X_val))

0.28651301818876285

Using the full training set gave a slightly better score yet for the validation set.

# Optimizing income

The yearly income of a listing is the price * nights_booked. To optimize the income, I will use the model to predict the nights booked and then calculate the yearly income for various prices.

In [18]:
pipe = make_pipeline(ohe_processor, RandomForestRegressor(random_state=42, n_estimators=500))
pipe.set_params(**rfr_search.best_params_)
pipe.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(handle_unknown='infrequent_if_exist'),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7b10fd75d2d0>)])),
                ('randomforestregressor',
                 RandomForestRegressor(max_depth=12, max_features='sqrt',
                                       n_estimators=500, random_state=42))])

In [ ]:
# Function to add best price, bookings, and revenue to a DataFrame
def optimize_income(x):
    x = x.copy()
    # This minimum and max price seem reasonable to capture most of the data
    price_min = 20
    price_max = 1000
    best_revenue = 0
    best_price = 0
    best_bookings = 0
    # Loop through prices and calculate revenue
    # Use log scale for price because price differences are less impactful at higher prices
    for price in np.linspace(price_min, price_max, 100):
        x['price'] = price
        x['log_price'] = np.log10(price)
        x['price^2'] = price ** 2
        x['price^3'] = price ** 3
        x['price^4'] = price ** 4
        predicted_bookings = 10 ** pipe.predict(x.to_frame().T)
        projected_revenue = price * predicted_bookings
        if projected_revenue > best_revenue:
            best_revenue = projected_revenue
            best_price = price
            best_bookings = predicted_bookings
    return best_revenue, best_price, best_bookings

In [22]:
X_val_opt  = X_val.sample(20).copy()
X_val_opt['best_revenue'], X_val_opt['best_price'], X_val_opt['best_bookings'] = zip(*X_val_opt.progress_apply(optimize_income, axis=1))
X_val_opt

100%|██████████| 20/20 [00:57<00:00,  2.88s/it]


,neighbourhood,latitude,longitude,room_type,price,log_days_since_last_review,log_reviews_per_month,log_number_of_reviews,log_price,log_minimum_nights,log_host_listings_count,host_listings_minimum_nights,price^2,price^3,price^4,best_revenue,best_price,best_bookings
67,West Town,41.89355,-87.66210,Entire home/apt,129,2.090258,0.342423,1.756636,2.110590,0.477121,0.477121,0.227645,16641,2146689,276922881,[33643.98860631443],821.818182,[40.93848171122331]
849,Loop,41.87227,-87.62412,Entire home/apt,140,2.220370,0.773786,1.892651,2.146128,0.477121,1.672098,0.797793,19600,2744000,384160000,[71606.78686380049],841.616162,[85.08247599035343]
515,Edgewater,41.99610,-87.65794,Entire home/apt,53,2.267406,-0.537602,0.491362,1.724276,1.322219,0.000000,0.000000,2809,148877,7890481,[72295.5428892884],990.101010,[73.01835080636147]
424,West Town,41.89615,-87.67934,Private room,50,2.127429,-0.107905,1.892651,1.698970,0.477121,0.301030,0.143628,2500,125000,6250000,[36353.90819267616],901.010101,[40.347947433575555]
3,Portage Park,41.96769,-87.76821,Entire home/apt,60,-1.000000,-1.000000,-1.000000,1.778151,0.000000,1.000000,0.000000,3600,216000,12960000,[11576.477909323743],940.606061,[12.30746684947434]
415,Loop,41.88488,-87.63337,Entire home/apt,79,2.458033,-0.455932,0.322219,1.897627,1.491362,1.477121,2.202922,6241,493039,38950081,[18277.537268921118],1000.000000,[18.27753726892112]
290,Near West Side,41.86935,-87.65541,Entire home/apt,65,-1.000000,-1.000000,-1.000000,1.812913,1.146128,1.431364,1.640526,4225,274625,17850625,[26246.253099190235],990.101010,[26.508662077329454]
370,infrequent,41.86659,-87.71788,Entire home/apt,164,2.649432,-0.236572,1.004321,2.214844,0.301030,0.301030,0.090619,26896,4410944,723394816,[28047.879217608373],841.616162,[33.32621270455147]
327,Near North Side,41.89861,-87.63811,Entire home/apt,192,2.064832,0.252853,0.612784,2.283301,0.301030,1.491362,0.448945,36864,7077888,1358954496,[31123.495674279537],990.101010,[31.434667126644296]
31,Uptown,41.96936,-87.65956,Entire home/apt,123,2.029789,0.553883,1.919601,2.089905,0.477121,0.301030,0.143628,15129,1860867,228886641,[41957.800969185184],841.616162,[49.85384416645863]


Those results are a little depressing. It doesn't seem like the model is able to capture raises in price equating to less bookings or even no bookings. The target variable did have a very skewed distribution with many values being booked year round or not at all. I think my methods are sound and with the right data this kind of optimization could be very useful.

# Final Pipeline

In [23]:
X , y = pd.concat([X_train, X_val]), pd.concat([y_train, y_val])
pipe.fit(X, y)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(handle_unknown='infrequent_if_exist'),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7b10fd75d2d0>)])),
                ('randomforestregressor',
                 RandomForestRegressor(max_depth=12, max_features='sqrt',
                                       n_estimators=500, random_state=42))])

# Joblib dump

In [24]:
joblib.dump(pipe, 'model.joblib')

['model.joblib']

The optimize_income function will be saved in utils.py

**Continued in final_test.ipynb**